# MIDTERM: PARAMETER ESTIMATION

# Questions

In this homework, you will do a parameter estimation for
BioModels model 957, a model of COVID.

1. (10 pts) **Acquire the Model**. Create a roadrunner instance of the model and print the antimony representation of the model in an output cell.
1. (20 pts) **Data Generation**. Create synthetic data for the floating species using the values of the parameter in the published model. Use start time 0 and end time of 5.
   1. (10 pts) Create a function ``generateNoisyData``: Creates noisy versions of the simulation floating species. Inputs are antimony model and stadard deviation of noise; outputs are SBstoat.NamedTimeseries.
   1. (5 pts) Create and save in a global variable synthetic data for standard deviations of 0, 0.1, 0.5, 1.0, 2.0.
   1. (5 pts) Plot the three datatsets. What characteristics of the data confirm that these synthetic data
   are consistent with your expectations.
1. (40 pts) **Experiment Infrastructure**. Implement the following functions. Feel free to use ``SBstoat`` or ``lmfit``.:
    1. (10 pts) ``getParameters``: Creates a list of ``SBstoat.Parameter`` (or ``lmfit.Parameter``) from a dictionary with keys of parameter names and values that are the values of parameters in the model. It also has arguments that indicate the range for the parameter (as a fraction of the parameter's true value) and the starting value.
    1. (30 pts) ``doFit``: Encapsulates the fitting and evaluation workflow. Inputs are: antimony model, standard deviation of noise, the fitting method, and fractions of the true value for: lower end of parameter search, starting value of parameter search, and upper end of parameter search; Outputs are: residual sum of squares of the fit and the number of function evaluations (see ``getFitterInfo`` in Helper Functions.  
1. (30 pts) **Experiments and Analysis**.
Use the experiment infrastructure to determine of effects of: standard deviation of noise,
search range, and search algorithm. You want to understand the impact on accuracy of parameter estimates and the time to do the parameter estimation. We will quantify accuracy in terms of residual sum of square, and time is in units of function evaluations.
    1. (10 pts) Plot the results. (16 plots.) All experiments should set the starting value of the searchto the lower end of the range.
        1. Use the data generated in Question 2.
        1. Use the search ranges lower_frc, upper_frc = (0.5, 2.0) and (0.25, 4.0)
        1. Use the search algorithms ``leastsquares`` and ``differential_evolution`` Construct plots as follows:
            1. x-axis: standard deviation
            2. y-axis: residual sum of squares, maximum estimation error, median estimation error, number of function evaluations (set getFitterInfo in Helpers)
            3. Do 2 plots; one for each combination of lower_frc, upper_frc.
    1. (20 pts) Answer the following questions. (5 pts for each correct answer)
       1. Which algorithm (leastsquares or differential_evolution) works best for these data and why?
       1. Why are the maximum error and median error higher for the search range (0.25, 4.0) compared with (0.5, 2.0)?
       1. Why does the residual sum of squares (RSSQ) increase with standard deviation?
       1. Why does do estimation errors (max_err and median_err) increase from standard deviation of 0 to non-zero standard deviations?

**Please do your homework in a copy of this notebook, maintaining the sections.**

**Don't forget to document your functions and include tests. All tests must have at least one ``assert``**

**Feel free to make use of any of the codes in the Helper Functions or any codes we developed in class. However, you are not required to use these codes.**In this homework, you will do a parameter estimation for
BioModels model 957, a model of COVID.

1. (10 pts) **Acquire the Model**. Create a roadrunner instance of the model and print the antimony representation of the model in an output cell.

1. (20 pts) **Data Generation**. Create synthetic data for the floating species using the values of the parameter in the published model:
   1. (10 pts) Create a function ``generateNoisyData``: Creates noisy versions of the simulation floating species. Inputs are antimony model and stadard deviation of noise; outputs are SBstoat.NamedTimeseries.
   1. (5 pts) Create and save in a global variable synthetic data for standard deviations of 0, 0.1, 0.5, 1.0, 2.0.
   1. (5 pts) Plot the three datatsets. What characteristics of the data confirm that these synthetic data
   are consistent with your expectations.
   
1. (40 pts) **Experiment Infrastructure**. Implement the following functions:

    1. (10 pts) ``getParameters``: Creates a list of SBstoat.Parameter from a dictionary with keys of parameter names and values that are the values of parameters in the model. It also has arguments that indicate the range for the parameter (as a fraction of the parameter's true value) and the starting value.
    1. (30 pts) ``doFit``: Encapsulates the fitting and evaluation workflow. Inputs are: antimony model, standard deviation of noise, the fitting method, and fractions of the true value for: lower end of parameter search, starting value of parameter search, and upper end of parameter search; Outputs are: residual sum of squares of the fit and the number of function evaluations (see ``getFitterInfo`` in Helper Functions.

   
1. (30 pts) **Experiments and Analysis**.
Use the experiment infrastructure to determine of effects of: standard deviation of noise,
search range, and search algorithm. You want to understand the impact on accuracy of parameter estimates and the time to do the parameter estimation. We will quantify accuracy in terms of residual sum of square, and time is in units of function evaluations.

    A. (10 pts) Plot the results. All experiments should set the starting value of the search
to the lower end of the range.
        1. Use the data generated in Question 2.
        1. Use the search ranges lower_frc, upper_frc = (0.5, 2.0) and (0.25, 4.0)
        1. Use the search algorithms ``leastsquares`` and ``differential_evolution``
   
   Construct plots as follows:
   
        1. x-axis: standard deviation
        2. y-axis: residual sum of squares, maximum estimation error, median estimation error, number of function evaluations (set getFitterInfo in Helpers)
        3. Do 2 plots; one for each combination of lower_frc, upper_frc
    
   B. (20 pts) Answer the following questions. (5 pts for each correct answer)
       1. Which algorithm (leastsquares or differential_evolution) works best for these data and why?
       1. Why are the maximum error and median error higher for the search range (0.25, 4.0) compared with (0.5, 2.0)?
       1. Why does the residual sum of squares (RSSQ) increase with standard deviation?
       1. Why does do estimation errors (max_err and median_err) increase from standard deviation of 0 to non-zero standard deviations?

**Please do your homework in a copy of this notebook, maintaining the sections.**

**Don't forget to document your functions and include tests. All tests must have at least one ``assert``**

**Feel free to make use of any of the codes in the Helper Functions or any codes we developed in class. However, you are not required to use these codes.**

# Programming Preliminaries
This section provides the setup to run your python codes.

## Imports

In [ ]:
!pip install -q tellurium
!pip install -q SBstoat

In [1]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SBstoat
import tellurium as te

## Constants

In [2]:
STDS = [0, 0.1, 0.5, 1.0, 2.0]
# Key and column names
STD = "std"
RSSQ = "rssq"
NUM_EVAL = "num_eval"
METHOD = "method"
MAX_ERR = "max_err"
MEDIAN_ERR = "median_err"
REPL = "repl"
TS = "timeseries"

# Helper Functions

In [3]:
TEST_MODEL = """
A->B; A
B->; k*B; 
A = 10; 
B=0
k = 2
"""
TEST_MODEL_RR = te.loada(TEST_MODEL)
TEST_MODEL_ARR = TEST_MODEL_RR.simulate()
TEST_MODEL_TS = SBstoat.NamedTimeseries(array=TEST_MODEL_ARR, colnames=TEST_MODEL_ARR.colnames)

In [4]:
def arrToDF(arr):
    """
    Converts a NamedArray into a DataFrame.
    If it is simulation output, makes TIME the index.

    Parameters
    ----------
    arr: NamedArray
    
    Returns
    -------
    DataFrame
        Removes "[" , "]" from the names of species
    """
    columns = [c[1:-1] if c[0] == "[" else c for c in arr.colnames]
    df = pd.DataFrame(arr, columns=columns)
    df = df.set_index("time")
    return df

# Tests
assert(isinstance(arrToDF(TEST_MODEL_ARR), pd.DataFrame))
print("OK!")

OK!


In [5]:
def arrToTS(arr):
    """
    Converts NamedArray to a NamedTimeseries
    
    Parameters
    ----------
    arr: NamedArray
    
    Returns
    -------
    NamedTimeseries
    """
    return SBstoat.NamedTimeseries(array=arr, colnames=arr.colnames)

# Tests
assert(isinstance(arrToTS(TEST_MODEL_ARR), SBstoat.NamedTimeseries))
print("OK!")

OK!


In [6]:
def getParameterValueDct(model):
    """
    Finds the name and value of all parameters.
    
    Parameters
    ----------
    model: str (Antimony model)
    
    Returns
    -------
    dict (key: str, value: float)
    """
    rr = te.loada(model)
    return {n: rr[n] for n in rr.model.getGlobalParameterIds()}

# Tests
dct = getParameterValueDct(TEST_MODEL)
assert(isinstance(dct, dict))
assert("k" in dct.keys())
print("OK!")

OK!


In [7]:
FitterInfo = collections.namedtuple("FitterInfo", "rssq num_eval fitter max_err median_err")

def getFitterInfo(fitter):
    """
    Calculates statistics for the accuracy of a fit.
    
    Parameters
    ----------
    fitter: SBstoat.ModelFitter
    
    Returns
    -------
    FitterInfo
    """
    df = (fitter.residualsTS.to_dataframe()**2)
    rssq = df.sum().sum()
    # Find the number of function evaluations
    report_stg = fitter.reportFit()
    pos = report_stg.index("function evals")
    stg = report_stg[pos:]
    start_pos = stg.index("=") + 1
    end_pos = stg.index("\n")
    num_eval = int(stg[start_pos:end_pos])
    # Get statistics on the fit
    fitter.roadrunnerModel.resetAll()
    parameter_value_dct = getParameterValueDct(fitter.roadrunnerModel.getAntimony())
    errs = []
    for key, value in fitter.params.valuesdict().items():
        err = np.abs(value - parameter_value_dct[key])/parameter_value_dct[key]
        errs.append(err)
    max_err = max(errs)
    median_err = np.median(errs)
    #
    return FitterInfo(rssq=rssq, num_eval=num_eval, fitter=fitter, max_err=max_err, median_err=median_err)
    

# Tests
fitter = SBstoat.ModelFitter(TEST_MODEL, TEST_MODEL_TS, parametersToFit=[SBstoat.Parameter("k", lower=1, value=1, upper=4)])
fitter.fitModel()
fitter_info = getFitterInfo(fitter)
assert(np.isclose(fitter_info.rssq, 0))
assert(fitter_info.num_eval < 10)
print("OK!")

OK!


# Question 1: Acquire the model

# Question 2: Data Generation

# Question 3: Experiment Infrastructure

# Question 4: Experiment Execution and Analysis